## Make Profile summaries using Jinja2 and Python Modele

- Fetch SD file from IG
- Transform to Python model
- use Jinja2 template to create a summary markdown file
- save markdown file



### import python modules including R4 fhirclient models

In [16]:
from fhirclient.r4models import structuredefinition as SD
from fhirclient.r4models import narrative as N
from fhirclient.r4models import valueset as VS
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
import fhirclient.r4models.fhirreference as FR
from json import dumps, loads, load
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from pprint import pprint
from stringcase import snakecase, titlecase
from collections import namedtuple
from pandas import *
from datetime import date
from jinja2 import Environment, FileSystemLoader, select_autoescape
from commonmark import commonmark
from IPython.display import display, HTML, Markdown
from lxml import etree

### Get file and return as dict and generate SD model instance

In [17]:
#in_path = '/Users/ehaas/Documents/FHIR/US-Core-R4/output/StructureDefinition-'
#f_name = 'us-core-inspired-oxygen'
in_path = '/Users/ehaas/.fhir/packages/hl7.fhir.us.core.r4#dev/package'
files = [x for x in os.listdir(in_path) if x.startswith("StructureDefinition")]

files

['StructureDefinition-us-core-careteam.json',
 'StructureDefinition-us-core-medicationstatement.json',
 'StructureDefinition-us-core-direct.json',
 'StructureDefinition-us-core-profile-link.json',
 'StructureDefinition-us-core-device.json',
 'StructureDefinition-us-core-practitioner.json',
 'StructureDefinition-us-core-patient.json',
 'StructureDefinition-us-core-immunization.json',
 'StructureDefinition-us-core-smokingstatus.json',
 'StructureDefinition-us-core-documentreference.json',
 'StructureDefinition-new-us-core-diagnosticreport.json',
 'StructureDefinition-us-core-race.json',
 'StructureDefinition-us-core-medicationrequest.json',
 'StructureDefinition-us-core-diagnosticreport.json',
 'StructureDefinition-us-core-condition.json',
 'StructureDefinition-us-core-birthsex.json',
 'StructureDefinition-us-core-encounter.json',
 'StructureDefinition-us-core-organization.json',
 'StructureDefinition-us-core-pediatric-weight.json',
 'StructureDefinition-us-core-procedure.json',
 'Struct

In [18]:
def open_file(in_path, f_name): # get files
    with open(f'{in_path}/{f_name}') as f:
        r = f.read()
        return(loads(r))
   

In [19]:
 
def expand(vs):
    print(vs)
    fhir_test_server = 'http://fhirtest.uhn.ca/baseDstu3'
    fhir_test_server = 'http://test.fhir.org/r4'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 
    params = dict(
          url = vs.split('|')[0],
          count = 20,
            )
    
    r = get(f'{fhir_test_server}/ValueSet/$expand', params = params, headers = headers )
    # return r.status_code
    # view  output

    
    if r.status_code == 200:
        print(r.status_code)
        return VS.ValueSet(r.json())

In [23]:
j_list = []    
for i in files:
    sd_dict = open_file(in_path,i)
    sd = SD.StructureDefinition(sd_dict)
    diff_e = [i.path for i in sd.differential.element]
    for j in sd.snapshot.element:
        if j.min == 1 and j.binding and j.path in diff_e:
            j_url = f'https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-{j.path.split(".")[0].lower()}-definitions.html#{j.path}'
            vs = expand(j.binding.valueSet)
            j_list.append((j,vs,j_url))
#j_list

http://hl7.org/fhir/ValueSet/care-team-status
200
http://hl7.org/fhir/us/core/ValueSet/us-core-careteam-provider-roles
http://hl7.org/fhir/ValueSet/medication-statement-status
200
http://hl7.org/fhir/us/core/ValueSet/us-core-medication-codes
http://hl7.org/fhir/ValueSet/device-kind
200
http://hl7.org/fhir/ValueSet/contact-point-system
200
http://hl7.org/fhir/ValueSet/administrative-gender
200
http://hl7.org/fhir/us/core/ValueSet/simple-language
http://hl7.org/fhir/ValueSet/immunization-status
200
http://hl7.org/fhir/us/core/ValueSet/us-core-cvx
http://hl7.org/fhir/ValueSet/observation-status
200
http://hl7.org/fhir/ValueSet/observation-codes
http://hl7.org/fhir/us/core/ValueSet/us-core-observation-ccdasmokingstatus
http://hl7.org/fhir/ValueSet/document-reference-status
200
http://hl7.org/fhir/ValueSet/c80-doc-typecodes
200
http://hl7.org/fhir/us/core/ValueSet/us-core-documentreference-category
http://hl7.org/fhir/ValueSet/mimetypes|4.0.0
http://hl7.org/fhir/ValueSet/diagnostic-report-s

### Using Jinja2 Template create md file for summary view

In [46]:
in_path = ''
template_path = 'summary_template.j2'

def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

env.filters['markdown'] = markdown

template = env.get_template(template_path)

d = template.render(elements = j_list)
display(Markdown(d))

## Sample Summary Markdown Template...

### All Required Coded Elements:


1. **[CareTeam.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-careteam-definitions.html#CareTeam.status)**
   - **required**
   - http://hl7.org/fhir/ValueSet/care-team-status
     (
     proposed, 
     active, 
     suspended, 
     inactive, 
     entered-in-error)
1. **[CareTeam.participant.role](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-careteam-definitions.html#CareTeam.participant.role)**
   - extensible
   - max binding = true
   - http://hl7.org/fhir/us/core/ValueSet/us-core-careteam-provider-roles
1. **[MedicationStatement.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-medicationstatement-definitions.html#MedicationStatement.status)**
   - **required**
   - http://hl7.org/fhir/ValueSet/medication-statement-status
     (
     active, 
     completed, 
     entered-in-error, 
     intended, 
     stopped, 
     on-hold, 
     **unknown**, 
     not-taken)
1. **[MedicationStatement.medication[x]](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-medicationstatement-definitions.html#MedicationStatement.medication[x])**
   - extensible
   - http://hl7.org/fhir/us/core/ValueSet/us-core-medication-codes
1. **[Device.type](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-device-definitions.html#Device.type)**
   - extensible
   - http://hl7.org/fhir/ValueSet/device-kind
     (
     156009, 
     271003, 
     287000, 
     291005, 
     678001, 
     739006, 
     793009, 
     882002, 
     972002, 
     989005, 
     994005, 
     1211003, 
     1333003, 
     1422002, 
     1579007, 
     1766001, 
     1941006, 
     1962007, 
     2248009, 
     2282003)
1. **[Patient.telecom.system](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-patient-definitions.html#Patient.telecom.system)**
   - **required**
   - http://hl7.org/fhir/ValueSet/contact-point-system
     (
     phone, 
     fax, 
     email, 
     pager, 
     url, 
     sms, 
     other)
1. **[Patient.gender](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-patient-definitions.html#Patient.gender)**
   - **required**
   - http://hl7.org/fhir/ValueSet/administrative-gender
     (
     male, 
     female, 
     other, 
     **unknown**)
1. **[Patient.communication.language](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-patient-definitions.html#Patient.communication.language)**
   - extensible
   - max binding = true
   - http://hl7.org/fhir/us/core/ValueSet/simple-language
1. **[Immunization.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-immunization-definitions.html#Immunization.status)**
   - **required**
   - http://hl7.org/fhir/ValueSet/immunization-status
     (
     completed, 
     entered-in-error, 
     not-done)
1. **[Immunization.vaccineCode](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-immunization-definitions.html#Immunization.vaccineCode)**
   - extensible
   - max binding = true
   - http://hl7.org/fhir/us/core/ValueSet/us-core-cvx
1. **[Observation.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-observation-definitions.html#Observation.status)**
   - **required**
   - http://hl7.org/fhir/ValueSet/observation-status
     (
     registered, 
     preliminary, 
     final, 
     amended, 
     corrected, 
     cancelled, 
     entered-in-error, 
     **unknown**)
1. **[Observation.code](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-observation-definitions.html#Observation.code)**
   - example
   - http://hl7.org/fhir/ValueSet/observation-codes
1. **[Observation.valueCodeableConcept](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-observation-definitions.html#Observation.valueCodeableConcept)**
   - extensible
   - max binding = true
   - http://hl7.org/fhir/us/core/ValueSet/us-core-observation-ccdasmokingstatus
1. **[DocumentReference.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-documentreference-definitions.html#DocumentReference.status)**
   - **required**
   - http://hl7.org/fhir/ValueSet/document-reference-status
     (
     current, 
     superseded, 
     entered-in-error)
1. **[DocumentReference.type](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-documentreference-definitions.html#DocumentReference.type)**
   - **required**
   - http://hl7.org/fhir/ValueSet/c80-doc-typecodes
     (
     55107-7, 
     74155-3, 
     51851-4, 
     67851-6, 
     34744-3, 
     34873-0, 
     68552-9, 
     67852-4, 
     68471-2, 
     68483-7, 
     64058-1, 
     64070-6, 
     64053-2, 
     64054-0, 
     34862-3, 
     64062-3, 
     64078-9, 
     64066-4, 
     64060-7, 
     64074-8)
1. **[DocumentReference.category](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-documentreference-definitions.html#DocumentReference.category)**
   - extensible
   - http://hl7.org/fhir/us/core/ValueSet/us-core-documentreference-category
1. **[DocumentReference.content.attachment.contentType](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-documentreference-definitions.html#DocumentReference.content.attachment.contentType)**
   - **required**
   - http://hl7.org/fhir/ValueSet/mimetypes|4.0.0
1. **[DiagnosticReport.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-diagnosticreport-definitions.html#DiagnosticReport.status)**
   - **required**
   - http://hl7.org/fhir/ValueSet/diagnostic-report-status
     (
     registered, 
     partial, 
     preliminary, 
     final, 
     amended, 
     corrected, 
     appended, 
     cancelled, 
     entered-in-error, 
     **unknown**)
1. **[DiagnosticReport.category](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-diagnosticreport-definitions.html#DiagnosticReport.category)**
   - extensible
   - http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-category
1. **[DiagnosticReport.code](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-diagnosticreport-definitions.html#DiagnosticReport.code)**
   - extensible
   - http://hl7.org/fhir/ValueSet/report-codes
1. **[Extension.extension.valueCoding](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-extension-definitions.html#Extension.extension.valueCoding)**
   - **required**
   - http://hl7.org/fhir/us/core/ValueSet/omb-race-category
1. **[Extension.extension.valueCoding](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-extension-definitions.html#Extension.extension.valueCoding)**
   - **required**
   - http://hl7.org/fhir/us/core/ValueSet/detailed-race
1. **[MedicationRequest.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-medicationrequest-definitions.html#MedicationRequest.status)**
   - **required**
   - http://hl7.org/fhir/ValueSet/medicationrequest-status
     (
     active, 
     on-hold, 
     cancelled, 
     completed, 
     entered-in-error, 
     stopped, 
     draft, 
     **unknown**)
1. **[MedicationRequest.medication[x]](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-medicationrequest-definitions.html#MedicationRequest.medication[x])**
   - extensible
   - http://hl7.org/fhir/us/core/ValueSet/us-core-medication-codes
1. **[DiagnosticReport.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-diagnosticreport-definitions.html#DiagnosticReport.status)**
   - **required**
   - http://hl7.org/fhir/ValueSet/diagnostic-report-status
     (
     registered, 
     partial, 
     preliminary, 
     final, 
     amended, 
     corrected, 
     appended, 
     cancelled, 
     entered-in-error, 
     **unknown**)
1. **[DiagnosticReport.category](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-diagnosticreport-definitions.html#DiagnosticReport.category)**
   - example
   - http://hl7.org/fhir/ValueSet/diagnostic-service-sections
     (
     AU, 
     BG, 
     BLB, 
     CG, 
     CH, 
     CP, 
     CT, 
     CTH, 
     CUS, 
     EC, 
     EN, 
     GE, 
     HM, 
     ICU, 
     IMG, 
     IMM, 
     LAB, 
     MB, 
     MCB, 
     MYC)
1. **[DiagnosticReport.code](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-diagnosticreport-definitions.html#DiagnosticReport.code)**
   - extensible
   - http://hl7.org/fhir/ValueSet/report-codes
1. **[Condition.verificationStatus](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-condition-definitions.html#Condition.verificationStatus)**
   - **required**
   - http://hl7.org/fhir/ValueSet/condition-ver-status
     (
     unconfirmed, 
     provisional, 
     differential, 
     confirmed, 
     refuted, 
     entered-in-error)
1. **[Condition.category](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-condition-definitions.html#Condition.category)**
   - preferred
   - http://hl7.org/fhir/us/core/ValueSet/us-core-condition-category
1. **[Condition.code](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-condition-definitions.html#Condition.code)**
   - extensible
   - http://hl7.org/fhir/us/core/ValueSet/us-core-problem
1. **[Encounter.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-encounter-definitions.html#Encounter.status)**
   - **required**
   - http://hl7.org/fhir/ValueSet/encounter-status|4.0.0
     (
     planned, 
     arrived, 
     triaged, 
     in-progress, 
     onleave, 
     finished, 
     cancelled, 
     entered-in-error, 
     **unknown**)
1. **[Encounter.class](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-encounter-definitions.html#Encounter.class)**
   - extensible
   - http://terminology.hl7.org/ValueSet/v3-ActEncounterCode
     (
     AMB, 
     EMER, 
     FLD, 
     HH, 
     IMP, 
     ACUTE, 
     NONAC, 
     OBSENC, 
     PRENC, 
     SS, 
     VR)
1. **[Encounter.type](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-encounter-definitions.html#Encounter.type)**
   - extensible
   - http://hl7.org/fhir/us/core/ValueSet/us-core-encounter-type
1. **[Observation.code](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-observation-definitions.html#Observation.code)**
   - extensible
   - http://hl7.org/fhir/ValueSet/observation-vitalsignresult
     (
     85353-1, 
     9279-1, 
     8867-4, 
     2708-6, 
     8310-5, 
     8302-2, 
     9843-4, 
     29463-7, 
     39156-5, 
     85354-9, 
     8480-6, 
     8462-4, 
     8478-0)
1. **[Procedure.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-procedure-definitions.html#Procedure.status)**
   - **required**
   - http://hl7.org/fhir/ValueSet/event-status
     (
     preparation, 
     in-progress, 
     not-done, 
     on-hold, 
     stopped, 
     completed, 
     entered-in-error, 
     **unknown**)
1. **[Procedure.code](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-procedure-definitions.html#Procedure.code)**
   - extensible
   - max binding = true
   - http://hl7.org/fhir/us/core/ValueSet/us-core-procedure-code
1. **[Medication.code](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-medication-definitions.html#Medication.code)**
   - extensible
   - http://hl7.org/fhir/us/core/ValueSet/us-core-medication-codes
1. **[Observation.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-observation-definitions.html#Observation.status)**
   - **required**
   - http://hl7.org/fhir/ValueSet/observation-status
     (
     registered, 
     preliminary, 
     final, 
     amended, 
     corrected, 
     cancelled, 
     entered-in-error, 
     **unknown**)
1. **[Observation.category](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-observation-definitions.html#Observation.category)**
   - preferred
   - http://hl7.org/fhir/ValueSet/observation-category
     (
     social-history, 
     vital-signs, 
     imaging, 
     laboratory, 
     procedure, 
     survey, 
     exam, 
     therapy, 
     activity)
1. **[Observation.code](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-observation-definitions.html#Observation.code)**
   - extensible
   - http://hl7.org/fhir/ValueSet/observation-codes
1. **[PractitionerRole.code](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-practitionerrole-definitions.html#PractitionerRole.code)**
   - extensible
   - http://hl7.org/fhir/us/core/ValueSet/us-core-provider-role
1. **[PractitionerRole.specialty](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-practitionerrole-definitions.html#PractitionerRole.specialty)**
   - extensible
   - http://hl7.org/fhir/us/core/ValueSet/us-core-provider-specialty
1. **[PractitionerRole.telecom.system](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-practitionerrole-definitions.html#PractitionerRole.telecom.system)**
   - **required**
   - http://hl7.org/fhir/ValueSet/contact-point-system|4.0.0
     (
     phone, 
     fax, 
     email, 
     pager, 
     url, 
     sms, 
     other)
1. **[DocumentReference.category](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-documentreference-definitions.html#DocumentReference.category)**
   - extensible
   - http://hl7.org/fhir/us/core/ValueSet/us-core-documentreference-category
1. **[CarePlan.text.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-careplan-definitions.html#CarePlan.text.status)**
   - **required**
   - http://hl7.org/fhir/us/core/ValueSet/us-core-narrative-status
1. **[CarePlan.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-careplan-definitions.html#CarePlan.status)**
   - **required**
   - http://hl7.org/fhir/ValueSet/request-status
     (
     draft, 
     active, 
     on-hold, 
     revoked, 
     completed, 
     entered-in-error, 
     **unknown**)
1. **[CarePlan.intent](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-careplan-definitions.html#CarePlan.intent)**
   - **required**
   - http://hl7.org/fhir/ValueSet/care-plan-intent
     (
     proposal, 
     plan, 
     order, 
     option)
1. **[CarePlan.category](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-careplan-definitions.html#CarePlan.category)**
   - example
   - http://hl7.org/fhir/ValueSet/care-plan-category
1. **[Goal.lifecycleStatus](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-goal-definitions.html#Goal.lifecycleStatus)**
   - **required**
   - http://hl7.org/fhir/ValueSet/goal-status
     (
     proposed, 
     planned, 
     accepted, 
     active, 
     on-hold, 
     completed, 
     cancelled, 
     entered-in-error, 
     rejected)
1. **[Goal.description](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-goal-definitions.html#Goal.description)**
   - example
   - http://hl7.org/fhir/ValueSet/clinical-findings
     (
     109006, 
     122003, 
     127009, 
     129007, 
     134006, 
     140004, 
     144008, 
     147001, 
     150003, 
     151004, 
     162004, 
     165002, 
     168000, 
     171008, 
     172001, 
     175004, 
     177007, 
     179005, 
     181007, 
     183005)
1. **[Extension.extension.valueCoding](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-extension-definitions.html#Extension.extension.valueCoding)**
   - **required**
   - http://hl7.org/fhir/us/core/ValueSet/omb-ethnicity-category
1. **[Extension.extension.valueCoding](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-extension-definitions.html#Extension.extension.valueCoding)**
   - **required**
   - http://hl7.org/fhir/us/core/ValueSet/detailed-ethnicity
1. **[Observation.code](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-observation-definitions.html#Observation.code)**
   - extensible
   - http://hl7.org/fhir/ValueSet/observation-vitalsignresult
     (
     85353-1, 
     9279-1, 
     8867-4, 
     2708-6, 
     8310-5, 
     8302-2, 
     9843-4, 
     29463-7, 
     39156-5, 
     85354-9, 
     8480-6, 
     8462-4, 
     8478-0)
1. **[Observation.code](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-observation-definitions.html#Observation.code)**
   - extensible
   - http://hl7.org/fhir/ValueSet/observation-vitalsignresult
     (
     85353-1, 
     9279-1, 
     8867-4, 
     2708-6, 
     8310-5, 
     8302-2, 
     9843-4, 
     29463-7, 
     39156-5, 
     85354-9, 
     8480-6, 
     8462-4, 
     8478-0)
1. **[AllergyIntolerance.verificationStatus](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-allergyintolerance-definitions.html#AllergyIntolerance.verificationStatus)**
   - **required**
   - http://hl7.org/fhir/ValueSet/allergyintolerance-verification
     (
     unconfirmed, 
     confirmed, 
     refuted, 
     entered-in-error)
1. **[AllergyIntolerance.code](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-allergyintolerance-definitions.html#AllergyIntolerance.code)**
   - extensible
   - https://vsac.nlm.nih.gov/valueset/2.16.840.1.113762.1.4.1186.8/expansion

### All Max Bindings for Required Coded Elements:


1. **[CareTeam.participant.role](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-careteam-definitions.html#CareTeam.participant.role)**  
    - http://hl7.org/fhir/us/core/ValueSet/us-core-careteam-provider-roles
1. **[Patient.communication.language](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-patient-definitions.html#Patient.communication.language)**  
    - http://hl7.org/fhir/us/core/ValueSet/simple-language
1. **[Immunization.vaccineCode](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-immunization-definitions.html#Immunization.vaccineCode)**  
    - http://hl7.org/fhir/us/core/ValueSet/us-core-cvx
1. **[Observation.valueCodeableConcept](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-observation-definitions.html#Observation.valueCodeableConcept)**  
    - http://hl7.org/fhir/us/core/ValueSet/us-core-observation-ccdasmokingstatus
1. **[Procedure.code](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-procedure-definitions.html#Procedure.code)**  
    - http://hl7.org/fhir/us/core/ValueSet/us-core-procedure-code


### All Required Bindings for Required Coded Elements:


1. **[CareTeam.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-careteam-definitions.html#CareTeam.status)**  
    - http://hl7.org/fhir/ValueSet/care-team-status
    (
     proposed, 
     active, 
     suspended, 
     inactive, 
     entered-in-error)
1. **[MedicationStatement.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-medicationstatement-definitions.html#MedicationStatement.status)**  
    - http://hl7.org/fhir/ValueSet/medication-statement-status
    (
     active, 
     completed, 
     entered-in-error, 
     intended, 
     stopped, 
     on-hold, 
     **unknown**, 
     not-taken)
1. **[Patient.telecom.system](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-patient-definitions.html#Patient.telecom.system)**  
    - http://hl7.org/fhir/ValueSet/contact-point-system
    (
     phone, 
     fax, 
     email, 
     pager, 
     url, 
     sms, 
     other)
1. **[Patient.gender](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-patient-definitions.html#Patient.gender)**  
    - http://hl7.org/fhir/ValueSet/administrative-gender
    (
     male, 
     female, 
     other, 
     **unknown**)
1. **[Immunization.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-immunization-definitions.html#Immunization.status)**  
    - http://hl7.org/fhir/ValueSet/immunization-status
    (
     completed, 
     entered-in-error, 
     not-done)
1. **[Observation.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-observation-definitions.html#Observation.status)**  
    - http://hl7.org/fhir/ValueSet/observation-status
    (
     registered, 
     preliminary, 
     final, 
     amended, 
     corrected, 
     cancelled, 
     entered-in-error, 
     **unknown**)
1. **[DocumentReference.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-documentreference-definitions.html#DocumentReference.status)**  
    - http://hl7.org/fhir/ValueSet/document-reference-status
    (
     current, 
     superseded, 
     entered-in-error)
1. **[DocumentReference.type](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-documentreference-definitions.html#DocumentReference.type)**  
    - http://hl7.org/fhir/ValueSet/c80-doc-typecodes
    (
     55107-7, 
     74155-3, 
     51851-4, 
     67851-6, 
     34744-3, 
     34873-0, 
     68552-9, 
     67852-4, 
     68471-2, 
     68483-7, 
     64058-1, 
     64070-6, 
     64053-2, 
     64054-0, 
     34862-3, 
     64062-3, 
     64078-9, 
     64066-4, 
     64060-7, 
     64074-8)
1. **[DocumentReference.content.attachment.contentType](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-documentreference-definitions.html#DocumentReference.content.attachment.contentType)**  
    - http://hl7.org/fhir/ValueSet/mimetypes|4.0.0
1. **[DiagnosticReport.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-diagnosticreport-definitions.html#DiagnosticReport.status)**  
    - http://hl7.org/fhir/ValueSet/diagnostic-report-status
    (
     registered, 
     partial, 
     preliminary, 
     final, 
     amended, 
     corrected, 
     appended, 
     cancelled, 
     entered-in-error, 
     **unknown**)
1. **[Extension.extension.valueCoding](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-extension-definitions.html#Extension.extension.valueCoding)**  
    - http://hl7.org/fhir/us/core/ValueSet/omb-race-category
1. **[Extension.extension.valueCoding](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-extension-definitions.html#Extension.extension.valueCoding)**  
    - http://hl7.org/fhir/us/core/ValueSet/detailed-race
1. **[MedicationRequest.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-medicationrequest-definitions.html#MedicationRequest.status)**  
    - http://hl7.org/fhir/ValueSet/medicationrequest-status
    (
     active, 
     on-hold, 
     cancelled, 
     completed, 
     entered-in-error, 
     stopped, 
     draft, 
     **unknown**)
1. **[DiagnosticReport.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-diagnosticreport-definitions.html#DiagnosticReport.status)**  
    - http://hl7.org/fhir/ValueSet/diagnostic-report-status
    (
     registered, 
     partial, 
     preliminary, 
     final, 
     amended, 
     corrected, 
     appended, 
     cancelled, 
     entered-in-error, 
     **unknown**)
1. **[Condition.verificationStatus](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-condition-definitions.html#Condition.verificationStatus)**  
    - http://hl7.org/fhir/ValueSet/condition-ver-status
    (
     unconfirmed, 
     provisional, 
     differential, 
     confirmed, 
     refuted, 
     entered-in-error)
1. **[Encounter.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-encounter-definitions.html#Encounter.status)**  
    - http://hl7.org/fhir/ValueSet/encounter-status|4.0.0
    (
     planned, 
     arrived, 
     triaged, 
     in-progress, 
     onleave, 
     finished, 
     cancelled, 
     entered-in-error, 
     **unknown**)
1. **[Procedure.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-procedure-definitions.html#Procedure.status)**  
    - http://hl7.org/fhir/ValueSet/event-status
    (
     preparation, 
     in-progress, 
     not-done, 
     on-hold, 
     stopped, 
     completed, 
     entered-in-error, 
     **unknown**)
1. **[Observation.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-observation-definitions.html#Observation.status)**  
    - http://hl7.org/fhir/ValueSet/observation-status
    (
     registered, 
     preliminary, 
     final, 
     amended, 
     corrected, 
     cancelled, 
     entered-in-error, 
     **unknown**)
1. **[PractitionerRole.telecom.system](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-practitionerrole-definitions.html#PractitionerRole.telecom.system)**  
    - http://hl7.org/fhir/ValueSet/contact-point-system|4.0.0
    (
     phone, 
     fax, 
     email, 
     pager, 
     url, 
     sms, 
     other)
1. **[CarePlan.text.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-careplan-definitions.html#CarePlan.text.status)**  
    - http://hl7.org/fhir/us/core/ValueSet/us-core-narrative-status
1. **[CarePlan.status](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-careplan-definitions.html#CarePlan.status)**  
    - http://hl7.org/fhir/ValueSet/request-status
    (
     draft, 
     active, 
     on-hold, 
     revoked, 
     completed, 
     entered-in-error, 
     **unknown**)
1. **[CarePlan.intent](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-careplan-definitions.html#CarePlan.intent)**  
    - http://hl7.org/fhir/ValueSet/care-plan-intent
    (
     proposal, 
     plan, 
     order, 
     option)
1. **[Goal.lifecycleStatus](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-goal-definitions.html#Goal.lifecycleStatus)**  
    - http://hl7.org/fhir/ValueSet/goal-status
    (
     proposed, 
     planned, 
     accepted, 
     active, 
     on-hold, 
     completed, 
     cancelled, 
     entered-in-error, 
     rejected)
1. **[Extension.extension.valueCoding](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-extension-definitions.html#Extension.extension.valueCoding)**  
    - http://hl7.org/fhir/us/core/ValueSet/omb-ethnicity-category
1. **[Extension.extension.valueCoding](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-extension-definitions.html#Extension.extension.valueCoding)**  
    - http://hl7.org/fhir/us/core/ValueSet/detailed-ethnicity
1. **[AllergyIntolerance.verificationStatus](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-allergyintolerance-definitions.html#AllergyIntolerance.verificationStatus)**  
    - http://hl7.org/fhir/ValueSet/allergyintolerance-verification
    (
     unconfirmed, 
     confirmed, 
     refuted, 
     entered-in-error)

### write to file

In [47]:
out_path = ''
out_file = 'sample_summary.md'

def write_file(path, f_name, data): # write file
    with open(f'{path}{f_name}', 'w') as f:
        f.write(data)

write_file(out_path,out_file,d)

In [ ]:
'''
rendered = template.render(cs=cs, p_map=p_map)
#print(rendered)

parser = etree.XMLParser(remove_blank_text=True)
root = etree.fromstring(rendered, parser=parser)

div = (etree.tostring(root[1][0], encoding='unicode', method='html'))
narr = N.Narrative()
narr.status = 'generated'
narr.div = div
cs.text = narr
'''